In [ ]:
!pip install duckdb==0.5.1
!pip install pytorch_tabular[extra]
!pip install torch
!pip install lightning

In [ ]:
import sys,os
project_path = "/content/drive/MyDrive/Work/quantitative_research/"
#database_path = "/content/drive/MyDrive/Work/trading_updated/database/development/develop"

sys.path.append(project_path)
#from featurewiz import featurewiz
import random
import lightgbm as lgb
import numpy as np
import sklearn.metrics as mt
import duckdb
import pandas as pd
import data.utils.duckdb_utils as ddu
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random
import numpy as np
import pandas as pd
import os

In [ ]:
database_path = "/content/drive/Othercomputers/My Mac/google_drive/database/production/banknifty_final1.duckdb"
db_connection = duckdb.connect(database=database_path , read_only=False)
train_table = 'banknifty.train'
val_table = 'banknifty.validation'
buffer_table = 'banknifty.buffer'

In [ ]:
data_suffix = 'improved_one'
train_feature_table = f'banknifty.train_feature_{data_suffix}'
train_feature_info_table = f'banknifty.train_feature_info_{data_suffix}'
train_feature_selection_table = f'banknifty.train_feature_selection_{data_suffix}'
train_training_info_table = f'banknifty.train_training_info_{data_suffix}'
train_tuning_info_table = f'banknifty.train_tuning_info_{data_suffix}'
ohlcraw_table = 'banknifty.ohlc_raw_data'

train_feature_selection_table_df = ddu.load_table_df(db_connection,table_name=train_feature_selection_table,
                                             column_names=None,filter=None,load_sql=None)



In [ ]:
target = 'label__minus_15_1min_close__label_generator_4class'
cols = train_feature_selection_table_df[train_feature_selection_table_df['label_name']==target]['feature_name'].tolist()

In [ ]:
len(cols)

366

In [73]:
sql = f"Select count(*) as cnt, time_split from {train_table} group by time_split"
a = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
train_cols = [col for col in a['time_split'].tolist() if 'train' in str(col)]

sql = f"Select count(*) as cnt, time_split from {val_table} group by time_split"
a = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
val_cols = [col for col in a['time_split'].tolist() if 'validation' in str(col)]

sql = f"Select count(*) as cnt, time_split from {buffer_table} group by time_split"
a = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
buffer_cols = [col for col in a['time_split'].tolist() if 'buffer' in str(col)]


In [74]:
len(train_cols)

10

In [ ]:
from fastai.tabular.core import df_shrink_dtypes,df_shrink,add_datepart

In [ ]:
cat_cols = ['timestampYear',
 'timestampMonth',
 'timestampWeek',
 'timestampDay',
 'timestampDayofweek',
 'timestampDayofyear',
 'timestampIs_month_end',
 'timestampIs_month_start',
 'timestampIs_quarter_end',
 'timestampIs_quarter_start',
 'timestampIs_year_end',
 'timestampIs_year_start',
 'timestampElapsed']

In [ ]:
sql = f'''select {",".join(cols)},{target},timestamp from {train_table} where time_split in 
('{"','".join(random.sample(train_cols, 5)) }') and
{target} != 'unknown' 
'''

train = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
train = add_datepart (train, field_name ='timestamp', prefix=None, drop=True, time=False)

train= df_shrink(train)
for c in cat_cols:
  train[c] = train[c].astype(str)
  train[c] = train[c].astype('category')

In [ ]:
train.shape

(241896, 367)

In [ ]:
sql = f'''select {",".join(cols)},{target},timestamp  from {val_table} where time_split in ('{"','".join(random.sample(val_cols, 5)) }') and
{target} != 'unknown' 
'''
print(sql)
valid = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
valid = add_datepart (valid, field_name ='timestamp', prefix=None, drop=True, time=False)

valid= df_shrink(valid)
for c in cat_cols:
  valid[c] = valid[c].astype(str)
  valid[c] = valid[c].astype('category')
valid.shape

select ROLLSTT_120_2_MEANDIFF_close,ROLLSTT_45_2_MAXMIN_close20,ROLLRNK_EMA_15_ema_close_15_None,ROLLDIFF_0_0_super_diff13,ROLLRNK_EMA_45_ema_close10_60_None,ROLLSTT_120_2_MAXMIN_close,TRUERANGE_120_hlc,ROLLSTT_30_2_MAXMIN_close,ROLLSTT_15_2_MAXMIN_close,ROLLPCM_DCL_60_60_don4_30_None,ROLLRNK_EMA_45_ema_close10_15_None,ROLLPCM_DCU_30_30_don2_30_None,ROLLRNK_EMA_120_ema_close20_15_None,DIFFCOL_DCU_20_20_hl50_close,ROLLPCM_DCU_60_60_don4_60_None,ROLLSTT_60_2_MAXMIN_close20,ROLLPCM_SUPERTs_45_30_supertrend_close_30_None,ROLLPCM_DCL_30_30_don2_30_None,ROLLPCM_DCU_240_240_don5_15_None,DIFFCOL_DCL_60_60_don4_close,ADX_30_adx14,ROLLBIN_100_0_True_5_close,ROLLPCM_DCL_120_120_don9_60_None,ROLLPCM_SUPERTs_15_30_supertrend_close10_30_None,ROLLPCM_DCL_60_60_don13_60_None,ROLLPCM_DCM_30_30_don7_30_None,ROLLPCM_SUPERTs_60_30_supertrend_close10_30_None,ROLLPCM_DCL_60_60_don13_15_None,ROLLPCM_SUPERTs_60_30_supertrend_close10_60_None,ROLLPCM_DCM_120_120_don14_15_None,ROLLPCM_DCL_240_240_don5_60_None,RO

(86904, 367)

In [ ]:
sql = f'''select {",".join(cols)},{target},timestamp from {buffer_table} where time_split in ('{"','".join(random.sample(buffer_cols, 5)) }') and
{target} != 'unknown' 
'''
print(sql)
test = ddu.load_table_df(db_connection,table_name=None,column_names=None,filter=None,load_sql=sql)
test = add_datepart (test, field_name ='timestamp', prefix=None, drop=True, time=False)

test= df_shrink(test)
for c in cat_cols:
  test[c] = test[c].astype(str)
  test[c] = test[c].astype('category')
test.shape

In [ ]:
#cols = cols+cat_cols

In [ ]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig, ModelConfig
from pytorch_tabular.models import BaseModel
from pytorch_tabular.models.common.layers import Embedding1dLayer
from pytorch_tabular.models.common.heads import LinearHeadConfig
import torch
import torch.nn as nn
import torch.nn.functional as F
from omegaconf import DictConfig
from typing import Dict
from dataclasses import dataclass, field


# Define a Config class to hold the hyperparameters of the model
# We need to inherit ModelConfig so that default parameters like learning rate, head, head_config, etc. 
# are present in the config
@dataclass
class CustomModelConfig(ModelConfig):
    use_batch_norm: bool = True
        
# Define the core model as a pure PyTorch model
# The forward method takes in a tensor and outputs a tensor

class CustomModelBase(nn.Module):
    def __init__(
        self,
        hparams: DictConfig,
        **kwargs
    ):
        super().__init__()
        self.hparams = hparams # Save the config to be accessed in the model elsewhere
        self._build_network()
    
    def _build_network(self):
        #Continuous and Categorical Dimensions are precalculated and stored in the config (InferredConfig)
        inp_dim = self.hparams.embedded_cat_dim + self.hparams.continuous_dim
        self.linear_layer_1 = nn.Linear(inp_dim, 200)
        self.linear_layer_2 = nn.Linear(inp_dim+200, 70)
        self.linear_layer_3 = nn.Linear(inp_dim+70, 70)
        self.input_batch_norm = nn.BatchNorm1d(self.hparams.continuous_dim)
        if self.hparams.use_batch_norm:
            self.batch_norm_2 = nn.BatchNorm1d(inp_dim+200)
            self.batch_norm_3 = nn.BatchNorm1d(inp_dim+70)
        self.dropout = nn.Dropout(0.3)
        self.output_dim = 70
    
    def forward(self, x: torch.Tensor):
        inp = x
        x = F.relu(self.linear_layer_1(x))
        x = self.dropout(x)
        x = torch.cat([x,inp], 1)
        if self.hparams.use_batch_norm:
            x = self.batch_norm_1(x)
        x = F.relu(self.linear_layer_2(x))
        x = self.dropout(x)
        x = torch.cat([x,inp], 1)
        if self.hparams.use_batch_norm:
            x = self.batch_norm_3(x)
        x = self.linear_layer_3(x)
        return x
    
    # It is also good practice to bundle the embedding scheme along with the core model
    # This is so that the model encapsulates it's requirement of how to embed the input
    # dictionary of categorical and continuous tensors and how to combine them into a single tensor.
    # We can either use one of the predefined embedding layers in PyTorch Tabular
    # Or define a custom embedding layer as well. Check Embedding1dLayer implementation 
    # on how to implement an embedding layer
    def _build_embedding_layer(self):
        return Embedding1dLayer(
            continuous_dim=self.hparams.continuous_dim,
            categorical_embedding_dims=self.hparams.embedding_dims,
            embedding_dropout=self.hparams.embedding_dropout,
            batch_norm_continuous_input=self.hparams.batch_norm_continuous_input,
        )

# Define the PyTorch Tabular model by inheriting BaseModel

class BaseModel(CustomModelBase):
    def __init__(
        self,
        config: DictConfig,
        **kwargs
    ):
        # Save any attribute that you need in _build_network before calling super()
        # After the super() call, the config will be saved as `hparams`
        super().__init__(config, **kwargs)

    def _build_network(self):
        # Backbone - Initialize the PyTorch model we defined earlier
        self._backbone = CustomModelBase(self.hparams)
        # Initializing Embedding Layer using the method we defined in the backbone
        self._embedding_layer = self._backbone._build_embedding_layer()
        # Head - we can use the helper method to initialize standard linear head
        self.head = self._get_head_from_config()
    
    # Now define the property methods which the BaseModel requires you to override
    @property
    def backbone(self):
        return self._backbone

    @property
    def embedding_layer(self):
        return self._embedding_layer

    @property
    def head(self):
        return self._head
    
    # For more customizations, we can override forward, compute_backbone, compute_head etc.

In [ ]:
def print_metrics(y_true, y_pred, tag):
    if isinstance(y_true, pd.DataFrame) or isinstance(y_true, pd.Series):
        y_true = y_true.values
    if isinstance(y_pred, pd.DataFrame) or isinstance(y_pred, pd.Series):
        y_pred = y_pred.values
    if y_true.ndim>1:
        y_true=y_true.ravel()
    if y_pred.ndim>1:
        y_pred=y_pred.ravel()
    val_acc = accuracy_score(y_true, y_pred)
    val_f1 = f1_score(y_true, y_pred)
    print(f"{tag} Acc: {val_acc} | {tag} F1: {val_f1}")
    return val_acc, val_fi

In [ ]:
import torchmetrics as tt

In [ ]:
data_config = DataConfig(
    target=[target], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=cols,
    categorical_cols=cat_cols)

trainer_config = TrainerConfig(
    auto_lr_find=True, # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=100,
    early_stopping="valid_loss", # Monitor valid_loss for early stopping
    early_stopping_mode = "min", # Set the mode as min because for val_loss, lower is better
    early_stopping_patience=5, # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss", # Save best checkpoint monitoring val_loss
    load_best=True, # After training, load the best checkpoint
#     accelerator="cpu"
)
optimizer_config = OptimizerConfig()

# DEPRECATED
# prediction head is defined separately now and head & head_config will be made
# mandatory in future releases
# model_config = CategoryEmbeddingModelConfig(
#     task="classification",
#     layers="1024-512-512",  # Number of nodes in each layer
#     activation="LeakyReLU", # Activation between each layers
#     learning_rate = 1e-3,
#     metrics=["f1","accuracy"], 
#     metrics_params=[{"num_classes":2},{}]
# )

head_config = LinearHeadConfig(
    layers="", # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

loss = tt.AveragePrecision(task="multiclass", num_classes=3, average=None)

model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="1024-512-512",  # Number of nodes in each layer
    activation="LeakyReLU", # Activation between each layers
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
    learning_rate = 1e-3,
    loss = loss,
    metrics=["f1_score","accuracy"], 
    metrics_params=[{"num_classes":3},{}]
)

In [ ]:
from pytorch_tabular.utils import get_balanced_sampler, get_class_weighted_cross_entropy
sampler = get_balanced_sampler(train[target].values.ravel())
weighted_loss = get_class_weighted_cross_entropy(train[target].values.ravel(), mu=0.04)


In [ ]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

2023-03-25 18:00:06,386 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off
INFO:pytorch_tabular.tabular_model:Experiment Tracking is turned off


In [ ]:
tabular_model.fit(train=train[cols+cat_cols+target], validation=valid[cols+cat_cols+target], 
                  #train_sampler=sampler, 
                  loss=weighted_loss)


INFO:lightning_lite.utilities.seed:Global seed set to 42
2023-03-25 18:00:07,086 - {pytorch_tabular.tabular_model:465} - INFO - Preparing the DataLoaders
INFO:pytorch_tabular.tabular_model:Preparing the DataLoaders
2023-03-25 18:00:07,449 - {pytorch_tabular.tabular_datamodule:286} - INFO - Setting up the datamodule for classification task
INFO:pytorch_tabular.tabular_datamodule:Setting up the datamodule for classification task
2023-03-25 18:00:52,380 - {pytorch_tabular.tabular_model:508} - INFO - Preparing the Model: CategoryEmbeddingModel
INFO:pytorch_tabular.tabular_model:Preparing the Model: CategoryEmbeddingModel
2023-03-25 18:00:52,480 - {pytorch_tabular.tabular_model:264} - INFO - Preparing the Trainer
INFO:pytorch_tabular.tabular_model:Preparing the Trainer
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False,

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=91` reached.
INFO:pytorch_lightning.tuner.lr_finder:LR finder stopped early after 91 steps due to diverging loss.
INFO:pytorch_lightning.tuner.lr_finder:Learning rate set to 0.000363078054770101
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at /content/.lr_find_6723d162-f4be-486b-bbbe-e46f50ad8e6d.ckpt
INFO:pytorch_lightning.utilities.rank_zero:Restored all states from the checkpoint file at /content/.lr_find_6723d162-f4be-486b-bbbe-e46f50ad8e6d.ckpt
2023-03-25 18:01:14,069 - {pytorch_tabular.tabular_model:560} - INFO - Suggested LR: 0.000363078054770101. For plot and detailed analysis, use `find_learning_rate` method.
INFO:pytorch_tabular.tabular_model:Suggested LR: 0.000363078054770101. For plot and detailed analysis, use `find_learning_rate` method.
2023-03-25 18:01:14,073 - {pytorch_tabular.tabular_model:566} - INFO - Training Started
INFO:pytorch_tabular.tabular_mode

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ custom_loss      │ CrossEntropyLoss          │      0 │
│ 1 │ _backbone        │ CategoryEmbeddingBackbone │  1.2 M │
│ 2 │ _embedding_layer │ Embedding1dLayer          │    732 │
│ 3 │ head             │ LinearHead                │  1.5 K │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 1.2 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.2 M                                                                                                
Total estimated model params size (MB): 4

Output()

2023-03-25 18:05:32,497 - {pytorch_tabular.tabular_model:568} - INFO - Training the model completed
INFO:pytorch_tabular.tabular_model:Training the model completed
2023-03-25 18:05:32,502 - {pytorch_tabular.tabular_model:1207} - INFO - Loading the best model
INFO:pytorch_tabular.tabular_model:Loading the best model


In [ ]:
result = tabular_model.evaluate(test[cols+cat_cols+target])


Output()

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test_accuracy       │    0.4726306200027466     │
│       test_f1_score       │    0.4726306200027466     │
│         test_loss         │    1.0384968519210815     │
└───────────────────────────┴───────────────────────────┘

In [ ]:
q = tabular_model.predict(test)

Output()

In [ ]:
q[['label__minus_15_1min_close__label_generator_4class','prediction']]

,label__minus_15_1min_close__label_generator_4class,prediction
0,put,put
1,put,call
2,put,call
3,put,call
4,put,call
...,...,...
64136,put,put
64137,put,put
64138,put,neutral
64139,put,neutral


In [ ]:
q['prediction'].value_counts()

neutral    36686
put        14284
call       13171
Name: prediction, dtype: int64

In [ ]:
q['label__minus_15_1min_close__label_generator_4class'].value_counts()

neutral    28721
call       17744
put        17676
Name: label__minus_15_1min_close__label_generator_4class, dtype: int64

In [ ]:
tabular_model.summary()

   | Name                                    | Type                      | Params
---------------------------------------------------------------------------------------
0  | custom_loss                             | CrossEntropyLoss          | 0     
1  | _backbone                               | CategoryEmbeddingBackbone | 1.2 M 
2  | _backbone.linear_layers                 | Sequential                | 1.2 M 
3  | _backbone.linear_layers.0               | Linear                    | 375 K 
4  | _backbone.linear_layers.1               | LeakyReLU                 | 0     
5  | _backbone.linear_layers.2               | Linear                    | 524 K 
6  | _backbone.linear_layers.3               | LeakyReLU                 | 0     
7  | _backbone.linear_layers.4               | Linear                    | 262 K 
8  | _backbone.linear_layers.5               | LeakyReLU                 | 0     
9  | _embedding_layer                        | Embedding1dLayer          | 732   
10 | _embe

In [ ]:
import sklearn.metrics as mt
c = mt.classification_report(q['label__minus_15_1min_close__label_generator_4class'],q['prediction'])

In [ ]:
print(c)

              precision    recall  f1-score   support

        call       0.35      0.26      0.29     17744
     neutral       0.56      0.72      0.63     28721
         put       0.37      0.30      0.33     17676

    accuracy                           0.47     64141
   macro avg       0.42      0.42      0.42     64141
weighted avg       0.45      0.47      0.45     64141



In [ ]:
c1 = mt.confusion_matrix(q['label__minus_15_1min_close__label_generator_4class'],q['prediction'])

In [ ]:
c1

array([[ 4553,  8356,  4835],
       [ 3946, 20544,  4231],
       [ 4672,  7786,  5218]])